In [ ]:
# Packages to be used
import networkx as nx
from networkx.algorithms import approximation
from networkx.algorithms import community
from networkx.algorithms import centrality
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import ndlib.models.epidemics as ep
import ndlib.models.ModelConfig as mc
import random
# import scipy
# import math
# import sys
import pandas as pd
# from fractions import Fraction
# import timeit
# from time import sleep
import igraph as ig
import time
import collections
# import heapq as heap
# from tqdm import tqdm 
import community
import json
from ndlib.viz.mpl.DiffusionTrend import DiffusionTrend

In [ ]:
# Load datasets, outside of definitions

G1 = nx.read_edgelist("Wiki-Vote.txt", create_using=nx.Graph(), nodetype= int)
# print(G1)
G2 = nx.read_edgelist("lastfm_asia_edges.txt", create_using=nx.Graph(), nodetype=int)
# G3 = nx.read_edgelist("facebook_combined.txt", create_using=nx.Graph(), nodetype=int)
G3 = nx.read_edgelist("musae_facebook_edges.txt", create_using=nx.Graph(), nodetype=int)
G4 = nx.read_edgelist("musae_ENGB_edges.txt", create_using=nx.Graph(), nodetype=int)



In [ ]:
print("The average shortest path length of the lastfm_asia dataset is,", nx.average_shortest_path_length(G1))

In [ ]:
avg_degree = sum(dict(nx.degree(G1)).values()) / len(G1)
print("The average degree of the Wiki-Vote dataset is,", sum(dict(nx.degree(G1)).values()) / len(G1))
print("The average degree of the lastfm_asia dataset is,",sum(dict(nx.degree(G2)).values()) / len(G2))
print("The average degree of the facebook dataset is,",sum(dict(nx.degree(G3)).values()) / len(G3))
print("The average degree of the twitch dataset is,",sum(dict(nx.degree(G4)).values()) / len(G4))

In [ ]:
print("The diameter of the Wiki-Vote dataset is, ",max([max(j.values()) for (i,j) in nx.shortest_path_length(G1)]))
print("The diameter of the lastfm_asia dataset is, ",max([max(j.values()) for (i,j) in nx.shortest_path_length(G2)]))
print("The diameter of the facebook dataset is, ",max([max(j.values()) for (i,j) in nx.shortest_path_length(G3)]))
print("The diameter of the twitch dataset is, ", max([max(j.values()) for (i,j) in nx.shortest_path_length(G4)]))

In [ ]:
# Load CSV file
filename = "musae_ENGB_edges.csv"
with open(filename, "r") as f:
    lines = f.readlines()

# Create empty graph
G = nx.Graph()

# Add edges to graph
for line in lines:
    parts = line.strip().split(",")
    if len(parts) == 2:
        src, dst = parts
        G.add_edge(src, dst)

# Print some information about the graph
print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())

In [ ]:
# The following can be done for any dataset that consists of files. First take the nodes and the attributes from the csv file, the name of
# which ends with "_target.csv". Then covnert the file ending with "_edges.csv" to a file "edges_txt". From this file extract the edges and
# add them to the created graph.

# Load CSV target file
filename = "musae_ENGB_target.csv"
delimiter = "," # or another character that separates columns
with open(filename, "r") as f:
    lines = f.readlines()

# Create graph
G1 = nx.Graph()

# Add nodes from file with attributes
for line in lines:
    fields = line.strip().split(delimiter)
    node_id = fields[0]
    attribute_value = fields[1]
    G1.add_node(node_id, attribute=attribute_value)

# Load TXT file with edge pairs
edge_file = "musae_ENGB_edges.txt"
edge_delimiter = "\t" # or another character that separates columns
with open(edge_file, "r") as f:
    edge_lines = f.readlines()

# Add edges to graph
for line in edge_lines:
    fields = line.strip().split(edge_delimiter)
    source = fields[0]
    target = fields[1]
    G1.add_edge(source, target)
    
print(G1)

In [ ]:
# Another way of reading both a txt file for the edges and then applyign the attributes to the nodes by using the json file.

G = nx.read_edgelist("musae_facebook_edges.txt", create_using=nx.Graph())

with open("musae_facebook_features.json") as f:
    data_facebook = json.load(f)
print(data_facebook)

nx.set_node_attributes(G, data_facebook, "attributes")
G.nodes["0"]["attributes"]

In [ ]:
from collections import defaultdict

# Get a dictionary of node attributes keyed by node
node_attrs = nx.get_node_attributes(G1, 'attribute')

# Group nodes based on their attributes using defaultdict
grouped_nodes = defaultdict(list)
for node, attr in node_attrs.items():
    grouped_nodes[attr].append(node)


In [ ]:
# Collect the nodes into groups according ot their attributes and then print then groups.
for attr, nodes in grouped_nodes.items():
    print(f"Nodes with attribute {attr}: {nodes}")

# Print the individual nodes to show their attributes
# for node, data in G1.nodes.data():
#     print(f"Node {node} has attribute value {data['attribute']}")

In [ ]:
for attr,nodes in grouped_nodes.items():
    print(f"The length of the group with attribute {attr} is {len(nodes)}")

In [ ]:
# Calculate the centrality measure
# dc = nx.degree_centrality(G1)
# print("Done with degree centrality")
# bc = nx.betweenness_centrality(G1)
# print("Done with betweenness centrality")
cc = nx.closeness_centrality(G1)
print("Done with closeness centrality")
# pagerank = nx.pagerank(G1)
# print("Done with pagerank")
# ec = nx.eigenvector_centrality(G1)
# print("Done with eigen vector")

In [ ]:
# sort_dc = sorted(dc.items(), key=lambda item: item[1], reverse=True)
# sort_bc = sorted(bc.items(), key=lambda item: item[1], reverse=True)
sort_cc = sorted(cc.items(), key=lambda item: item[1], reverse=True)
# sort_pc = sorted(pc.items(), key=lambda item: item[1], reverse=True)
# sort_ec = sorted(ec.items(), key=lambda item: item[1], reverse=True)

In [ ]:
only_keys= [k for k,v in sort_cc]
print(only_keys)

# Take the top 50 nodes to be used as seeds later
first_100 = only_keys[0:100]
print(first_100)

In [ ]:
# Create a copy of the original graph to be modified later, by removing nodes
G1_copy = G1.copy()


G1_copy.remove_nodes_from(first_100)
print(G1_copy)

In [ ]:
# Compute the total number of nodes in the graph
num_nodes = len(G1.nodes())

# Initialize a dictionary to store the number of seeds for each group
num_seeds = {}

# Calculate the total number of seed nodes as a fraction of the total number of nodes
# frac_seeds = 0.1
k = 100


# Iterate over the groups and calculate the number of seed nodes for each group
for attr, nodes in grouped_nodes.items():
    num_group_nodes = len(nodes)
    # num_group_seeds = int(num_group_nodes / num_nodes * frac_seeds)
    num_group_seeds = int((num_group_nodes / num_nodes) * k)

    num_seeds[attr] = num_group_seeds

# Sort the nodes in each group based on their centrality measure
for attr, nodes in grouped_nodes.items():
    # nodes.sort(key=lambda x: dc[x], reverse=True)
    # nodes.sort(key=lambda x: bc[x], reverse=True)
    nodes.sort(key=lambda x: cc[x], reverse=True)
    # nodes.sort(key=lambda x: pc[x], reverse=True)
    # nodes.sort(key=lambda x: ec[x], reverse=True)

# Initialize a list to store the seed nodes
seeds = []

# Iterate over the groups and add the top seed nodes to the list
for attr, nodes in grouped_nodes.items():
    num_group_seeds = num_seeds[attr]
    seeds += nodes[:num_group_seeds]

# Print the seed nodes
print(seeds)

In [ ]:
# Create a copy of the original graph to be modified later, by removing nodes
G1_copy = G1.copy()


G1_copy.remove_nodes_from(seeds)
print(G1_copy)

In [ ]:
from collections import defaultdict

# Get a dictionary of node attributes keyed by node
node_attrs = nx.get_node_attributes(G1_copy, 'attribute')

# Group nodes based on their attributes using defaultdict
grouped_nodes = defaultdict(list)
for node, attr in node_attrs.items():
    grouped_nodes[attr].append(node)

In [ ]:
# Calculate the centrality measure
dc_after = nx.degree_centrality(G1_copy)
print("Done with degree centrality")
bc_after = nx.betweenness_centrality(G1_copy)
print("Done with betweenness centrality")
cc_after = nx.closeness_centrality(G1_copy)
print("Done with closeness centrality")
pc_after = nx.pagerank(G1_copy)
print("Done with pagerank")
ec_after = nx.eigenvector_centrality(G1_copy)
print("Done with eigen vector")

In [ ]:
# Compute the total number of nodes in the graph
num_nodes_after = len(G1_copy.nodes())

# Initialize a dictionary to store the number of seeds for each group
num_seeds = {}

# Calculate the total number of seed nodes as a fraction of the total number of nodes
frac_seeds = 0.1
k = 100


# Iterate over the groups and calculate the number of seed nodes for each group
for attr, nodes in grouped_nodes.items():
    num_group_nodes = len(nodes)
    # num_group_seeds = int(num_group_nodes / num_nodes * frac_seeds)
    num_group_seeds = int((num_group_nodes / num_nodes_after) * k)

    num_seeds[attr] = num_group_seeds

# Sort the nodes in each group based on their centrality measure
for attr, nodes in grouped_nodes.items():
    # nodes.sort(key=lambda x: dc_after[x], reverse=True)
    # nodes.sort(key=lambda x: bc_after[x], reverse=True)
    # nodes.sort(key=lambda x: cc_after[x], reverse=True)
    # nodes.sort(key=lambda x: pc_after[x], reverse=True)
    nodes.sort(key=lambda x: ec_after[x], reverse=True)

# Initialize a list to store the seed nodes
seeds_after = []

# Iterate over the groups and add the top seed nodes to the list
for attr, nodes in grouped_nodes.items():
    num_group_seeds = num_seeds[attr]
    seeds_after += nodes[:num_group_seeds]

# Print the seed nodes
print(seeds_after)

In [ ]:
for node in G2.nodes:
    G2.nodes[node]['gender'] = 'male' if (node % 2 == 0) else 'female'

for node, attrs in G2.nodes(data=True):
    print(f"Node {node}: gender {attrs['gender']}")

male_nodes = [node for node,attr in G2.nodes(data='gender') if attr == 'male']
female_nodes = [n for n, attr in G2.nodes(data='gender') if attr == 'female']

bc = nx.betweenness_centrality(G2)

# calculate betweenness centralities for male and female nodes
male_bc = {n: bc[n] for n in male_nodes}
female_bc = {n: bc[n] for n in female_nodes}

print("Male nodes betweenness centralities:", male_bc)
print("Female nodes betweenness centralities:", female_bc)

Heuristics to block/remove nodes

In [ ]:
# # Centrality measures

# Degree centrality

degree_centrality = nx.centrality.degree_centrality(G2)
sort_degree_centrality = sorted(degree_centrality.items(), key=lambda item: item[1], reverse=True)
print("Done with degree centrality")

# Betweenness

# betweenness = nx.betweenness_centrality(G1)
# sort_betweenness = sorted(betweenness.items(), key=lambda item: item[1], reverse=True)
# print("Done with betweenness centrality")


# # # # Closeness

# closeness = nx.closeness_centrality(G1)
# sort_closeness = sorted(closeness.items(), key=lambda item: item[1], reverse=True)
# print("Done with closeness centrality")


# # # # Influence maximization
# Pagerank

# pagerank = nx.pagerank(G1)
# sort_pagerank = sorted(pagerank.items(), key=lambda item: item[1], reverse=True)
# print("Done with pagerank centrality")

# # Eigenvector

# eigenvector = nx.eigenvector_centrality(G1)
# sort_eigenvector = sorted(eigenvector.items(), key=lambda item: item[1], reverse=True)
# print("Done with eigenvector centrality")

In [ ]:
# To only get the keys from the sorted list

only_keys = [k for k,v in sort_pagerank]
print(only_keys)

# Take the top 50 nodes to be used as seeds later
first_100 = only_keys[0:1400]
print(first_100)

In [ ]:
# Add weight to the edges of the graph. To be used as thresholds for the ICM

degree_dict = dict(G2.degree())
weights = []
for u, v, data in G2.edges(data=True):
    ratio = 1 / degree_dict[v]
    data['weight'] = ratio
    weights.append(ratio)

print(weights)

In [ ]:
# Create a copy of the original graph to be modified later, by removing nodes
G1_copy = G1.copy()


G1_copy.remove_nodes_from(first_100)
print(G1_copy)

The following is the centrality measures for the new graph (After removing the nodes)

In [ ]:
# Centrality measures

# Degree centrality
degree_centrality_after = nx.centrality.degree_centrality(G1_copy)
sort_degree_centrality_after = sorted(degree_centrality_after.items(), key=lambda item: item[1], reverse=True)
print("Done with degree centrality")

# Betweenness

betweenness_after = nx.betweenness_centrality(G1_copy)
sort_betweenness_after = sorted(betweenness_after.items(), key=lambda item: item[1], reverse=True)
print("Done with betweenness centrality")

# Closeness

closeness_after = nx.closeness_centrality(G1_copy)
sort_closeness_after = sorted(closeness_after.items(), key=lambda item: item[1], reverse=True)
print("Done with closeness centrality")


# Influence maximization
# Pagerank

pagerank_after = nx.pagerank(G1_copy)
sort_pagerank_after = sorted(pagerank_after.items(), key=lambda item: item[1], reverse=True)
print("Done with pagerank centrality")

# Eigenvector

eigenvector_after = nx.eigenvector_centrality(G1_copy)
sort_eigenvector_after = sorted(eigenvector_after.items(), key=lambda item: item[1], reverse=True)
print("Done with eigenvector centrality")

In [ ]:
# To only get the keys from the sorted list

only_keys_after = [k for k,v in sort_pagerank_after]
print(only_keys_after)

# Take the top 50 nodes to be used as seeds later
first_100_after = only_keys_after[0:100]
print(first_100_after)

In [ ]:
# Adding weight to the edges of the graph

# for u,v in G1.edges():
#     G1[u][v]['weight'] = 1/G1.in_degree(v)
# G1.get_edge_data(30,1412)
# G1.in_degree(1412)

# for u,v in G2.edges():
#     G2[u][v]['weight'] = 1/G2.degree(v)
# G2.get_edge_data(0,5)

degree_dict = dict(G2_copy.degree())
weights = []
for u, v, data in G2_copy.edges(data=True):
    ratio = 1 / degree_dict[v]
    data['weight'] = ratio
    weights.append(ratio)

In [ ]:
# Adds an attribute to the nodes of the graph, for example degree centrality

nx.set_node_attributes(G1, centrality, "centrality")
# nx.set_node_attributes(G2, centrality, "centrality")

In [ ]:
# Filtering the graph by removing edges based on their weight attribute

for u,v in list(G1.edges()):
    if G1[u][v]['weight'] < 0.04:
        G1.remove_edge(u,v)
G1_new = G1.copy()

# Remove nodes based on their degree

for u in list(G1.nodes()):
    if G1.degree(u)< 10:
        G1.remove_node(u)

# Remove nodes based on their degree centrality

for u,v in centrality.items():
    if v < 0.004:
        G1.remove_node(u)

In [ ]:
# Lists of sources and targets based on in/out degrees of nodes

sources = []
targets = []
for u in G1.nodes():
    if G1.in_degree(u) == 0:
        sources.append(u)
for v in G1.nodes():
    if G1.out_degree(v) == 0:
        targets.append(v)

In [ ]:
# Splitting the sources to smaller components (useful for seed selection)

sub_sources = sources[0:50]
print(sub_sources)


In [ ]:
import random
def random(nodes, n):
    ''' Select seeds randomly
    Args:
        nodes (list) [#node]: node list of the graph;
        n (int): number of seeds;
    Returns: 
        seeds: (list) [#seed]: selected seed nodes index;
    '''
    # random.sample(nodes, n)
    # return nodes[:n]
    # np.random.shuffle(nodes)
    # return nodes[:n]
    seeds = random.sample(nodes, n)
    return seeds

In [ ]:
def mia(nodes, edges, n):
    ''' select seeds by mia policy
    args:
        nodes (list) [#node]: node list of the graph;
        edges (list of list) [#edge, 2]: edge list of the graph;
        n (int): number of seeds;
    returns: 
        seeds: (list) [#seed]: selected seed nodes index;
    '''
    out_connection = {}
    in_connection = {}
    centrality_score = {}
    seeds = []
    for edge in edges:
        if edge[0] in out_connection:
            out_connection[edge[0]].append(edge[1])
        else:
            out_connection[edge[0]] = [edge[1]]
        if edge[1] in in_connection:
            in_connection[edge[1]] += 1
        else:
            in_connection[edge[1]] = 1
    for node in nodes:
        centrality_score[node] = mia_centrality(node, out_connection, in_connection)
    i = 0
    for node, _ in sorted(centrality_score.items(), key=lambda item: item[1], reverse=True):
        if i >= n:
            break
        else:
            i += 1
        seeds.append(node)
    return seeds

def mia_centrality(node, out_connection, in_connection):
    ''' select seeds by mia centrality policy
    '''
    theta = 0.5
    c_score = 0
    q = 1  # threshold of influence
    visited = set()
    path_prob = 1
    c_score = dfs(visited, out_connection, path_prob, in_connection, node, theta, q)
    return c_score

def dfs(visited, out_connection, path_prob, in_connection, node, theta, q):
    if node not in visited:
        visited.add(node)
        if node in out_connection:
            for neighbour in out_connection[node]:
                path_prob *= q / in_connection[neighbour]
                if path_prob >= theta:
                    dfs(visited, out_connection, path_prob, in_connection, neighbour, theta, q)
                    path_prob /= (q / in_connection[neighbour])
                else:
                    path_prob /= (q / in_connection[neighbour])
    N_of_nodes = len(visited)
    return N_of_nodes

In [ ]:
def degree(edges, n):
    ''' Select seeds by degree policy
    Args:
        edges (list of list) [#edge, 2]: edge list of the graph;
        n (int): number of seeds;
    Returns: 
        seeds: (list) [#seed]: selected seed nodes index;
    '''
    degree = {}
    seeds = []
    for edge in edges:
        if edge[0] in degree:
            degree[edge[0]] += 1
        else:
            degree[edge[0]] = 1
        if edge[1] in degree:
            degree[edge[1]] += 1
        else:
            degree[edge[1]] = 1
    seeds = list({k: v for k, v in sorted(degree.items(), key=lambda item: item[1], reverse=True)}.keys())[:n]
    return seeds

In [ ]:
def degree_discount(edges, n):
    ''' Select seeds by degree discount degree
    Args:
        edges (list of list) [#edge, 2]: edge list of the graph;
        n (int): number of seeds;
    Returns: 
        seeds: (list) [#seed]: selected seed nodes index;
    '''
    out_degree = {}
    connection = {}
    seeds = []
    for edge in edges:
        if edge[1] in connection:
            connection[edge[1]].append(edge[0])
        else:
            connection[edge[1]] = [edge[0]]
        if edge[0] in out_degree:
            out_degree[edge[0]] += 1
        else:
            out_degree[edge[0]] = 1
    while len(seeds) < n:
        seed = sorted(out_degree.items(), key=lambda item: item[1], reverse=True)[0][0]
        seeds.append(seed)
        out_degree[seed] = -1
        if seed in connection:
            for node in connection[seed]:
                if node in out_degree:
                    out_degree[node] -= 1
    return seeds

In [ ]:
def degree_neighbor_fix(edges, n):
    ''' select seeds by degree neighbor fix policy
    args:
        edges (list of list) [#edge, 2]: edge list of the graph;
        n (int): number of seeds;
    returns: 
        seeds: (list) [#seed]: selected seed nodes index;
    '''
    out_degree = {}
    centrality_score = {}
    connection = {}
    seeds = []
    for edge in edges:
        if edge[1] in connection:
            connection[edge[1]].append(edge[0])
        else:
            connection[edge[1]] = [edge[0]]
        if edge[0] in out_degree:
            out_degree[edge[0]] += 1
        else:
            out_degree[edge[0]] = 1
    centrality_score = out_degree.copy()
    for edge in edges:
        if edge[1] in out_degree:
            centrality_score[edge[0]] += out_degree[edge[1]]
    while len(seeds) < n:
        seed = sorted(centrality_score.items(), key=lambda item: item[1], reverse=True)[0][0]
        seeds.append(seed)
        centrality_score[seed] = -1
        if seed in connection:
            for node in connection[seed]:
                if node in out_degree:
                    centrality_score[node] -= out_degree[seed]

    return seeds

In [ ]:
def degree_neighbor(edges, n):
    ''' select seeds by degree neighbor policy
    args:
        edges (list of list) [#edge, 2]: edge list of the graph;
        n (int): number of seeds;
    returns: 
        seeds: (list) [#seed]: selected seed nodes index;
    '''
    out_degree = {}
    centrality_score = {}
    seeds = []
    for edge in edges:
        if edge[0] in out_degree:
            out_degree[edge[0]] += 1
        else:
            out_degree[edge[0]] = 1
    centrality_score = out_degree.copy()
    for edge in edges:
        if edge[1] in out_degree:
            centrality_score[edge[0]] += out_degree[edge[1]]
    seeds = sorted(centrality_score.items(), key=lambda item: item[1], reverse=True)
    return seeds


In [ ]:
# Create communities based on the louvain method

louvain = nx.algorithms.community.louvain_communities(G2)
print(len(louvain))
# print(louvain[131])

In [ ]:
# Create a seed set based on the communities and a heuristic

seeds_new = {}
for community in louvain:
    max_node = max(community, key=lambda node: degree_centrality[node])
    seeds_new[max_node] = community

In [ ]:
# policy = 'degree'

# seeds_number = int(len(G1.nodes) * init_rate)
# seeds = list({k: v for k, v in sorted(dict(degreenew).items(), key=lambda item: item[1], reverse=True)}.keys())[:seeds_number]
policy = 'mia'

init_rate = 0.02
# nodes = G2_new.nodes
# edges = G2_new.edges
# # nodes = G_new.vs
# # edges = G_new.es
nodes = G2.nodes()
edges = G2.edges()
seeds_number = int(len(nodes) * init_rate)

if policy == 'degree':
    seeds = degree(edges, seeds_number)
    # seeds = list({k: v for k, v in sorted(dict(new_degree).items(), key=lambda item: item[1], reverse=True)}.keys())[:seeds_number]
if policy == 'random':
    seeds = random(nodes, seeds_number)
if policy == 'degree_discount':
    seeds = degree_discount(edges, seeds_number)
if policy == 'mia':
   seeds = mia(nodes, edges, seeds_number) 
if policy == 'degree_neighbor_fix':
    seeds = degree_neighbor_fix(edges, seeds_number)
if policy == 'degree_neighbor':
    seeds = degree_neighbor(edges, seeds_number)
print(f'Number of seeds: {len(seeds)}')

In [ ]:
print(seeds)

Use louvain communities without fairness measure and different centrality measures

In [ ]:
import community

# Partition into communities
partition = community.best_partition(G2)
for node in G2.nodes:
    G2.nodes[node]['community'] = partition[node]
print(len(partition))

num_distinct_values = len(set(partition.values()))

print("The dictionary has", num_distinct_values, "distinct values.")

In [ ]:
# Calculate centralities (can also be done from the cell above, have to check to decide), for fairness this cell can be used to removed the selected nodes

seeds = set()
for community_id in set(partition.values()):
    community_nodes = [node for node in G2.nodes if G2.nodes[node]['community'] == community_id]
    max_degree_node = max(community_nodes, key=lambda node: degree_centrality[node])
    seeds.add(max_degree_node)
print("Number of seeds:", len(seeds))

In [ ]:
# Create a copy of the original graph to be modified later, by removing nodes
G1_copy_comm = G1.copy()


G1_copy_comm.remove_nodes_from(seeds)
print(G1_copy_comm)

In [ ]:
# Centrality measures

# Degree
# sort_degree = sorted(dict(G2.degree).items(), key=lambda item: item[1], reverse=True)
# print(sort_degree)

# Degree centrality
degree_centrality_after = nx.centrality.degree_centrality(G1_copy_comm)
sort_degree_centrality_after = sorted(degree_centrality_after.items(), key=lambda item: item[1], reverse=True)
print("Degree centrality done")

# Betweenness

betweenness_after = nx.betweenness_centrality(G1_copy_comm)
sort_betweenness_after = sorted(betweenness_after.items(), key=lambda item: item[1], reverse=True)
print("Betweenness centrality done")

# Closeness

closeness_after = nx.closeness_centrality(G1_copy_comm)
sort_closeness_after = sorted(closeness_after.items(), key=lambda item: item[1], reverse=True)
print("Closeness centrality done")


# Influence maximization
# Pagerank

pagerank_after = nx.pagerank(G1_copy_comm)
sort_pagerank_after = sorted(pagerank_after.items(), key=lambda item: item[1], reverse=True)
print("Pagerank done")

# Eigenvector

eigenvector_after = nx.eigenvector_centrality(G1_copy_comm)
sort_eigenvector_after = sorted(eigenvector_after.items(), key=lambda item: item[1], reverse=True)
print("Eigenvector done")

In [ ]:
import community

# Partition into communities
partition = community.best_partition(G1_copy_comm)
for node in G1_copy_comm.nodes:
    G1_copy_comm.nodes[node]['community'] = partition[node]
print(len(partition))

num_distinct_values = len(set(partition.values()))

print("The dictionary has", num_distinct_values, "distinct values.")

In [ ]:
# Calculate centralities (can also be done from the cell above, have to check to decide)

seeds_after = set()
for community_id in set(partition.values()):
    community_nodes = [node for node in G1_copy_comm.nodes if G1_copy_comm.nodes[node]['community'] == community_id]
    max_degree_node = max(community_nodes, key=lambda node: pagerank_after[node])
    seeds_after.add(max_degree_node)
print("Number of seeds:", len(seeds_after))

The "correct" implementation for the equality fairness is bellow (as implemented in Farnandi's paper)

In [ ]:
import community
# Calculcate communities using louvain method
partition_original = community.best_partition(G2)
for node in G2.nodes:
    G2.nodes[node]['community'] = partition_original[node]
print(len(partition_original))

num_distinct_values = len(set(partition_original.values()))

print("The dictionary has", num_distinct_values, "distinct values.")

In [ ]:
# Create a dictionary to store the population ratio of each community:

community_sizes = {}
for c in set(partition_original.values()):
    nodes_in_community = [n for n in partition_original.keys() if partition_original[n] == c]
    community_sizes[c] = len(nodes_in_community)

# Calculate the centralities for each node in the graph

# closeness_centrality = nx.closeness_centrality(G2)
# betweenness_centrality = nx.betweenness_centrality(G2)
# degree_centrality = nx.degree_centrality(G2)
# eigen_centrality = nx.eigenvector_centrality(G2)
pagerank_centrality = nx.pagerank(G2)

# Create a dictionary to store the centralities for each community:

# community_closeness = {}
# community_betweenness = {}
# community_degree = {}
# community_eigen = {}
community_pagerank = {}
for node in G2.nodes():
    community = partition_original[node]
    # if community not in community_closeness:
    #     community_closeness[community] = 0
    # if community not in community_betweenness:
    #     community_betweenness[community] = 0
    # if community not in community_degree:
    #     community_degree[community] = 0
    # if community not in community_eigen:
    #     community_eigen[community] = 0
    if community not in community_pagerank:
        community_pagerank[community] = 0

    # community_closeness[community] += closeness_centrality[node]
    # community_betweenness[community] += betweenness_centrality[node]
    # community_degree[community] += degree_centrality[node]
    # community_eigen[community] += eigen_centrality[node]
    community_pagerank[community] += pagerank_centrality[node]


# Calculate the total centrality for each community

# for community in community_closeness:
#     community_closeness[community] /= community_sizes[community]
# for community in community_betweenness:
#     community_betweenness[community] /= community_sizes[community]
# for community in community_degree:
#     community_degree[community] /= community_sizes[community]
# for community in community_eigen:
#     community_eigen[community] /= community_sizes[community]
for community in community_pagerank:
    community_pagerank[community] /= community_sizes[community]

# Calculate the number of seeds for each community proportional to the population ratio using the centralities:
seeds = {}
for node in G2.nodes():
    community = partition_original[node]
    if community not in seeds:
        seeds[community] = []
        # if len(seeds[community]) < community_sizes[community] * community_closeness[community]:
        #     seeds[community].append(node)
        # if len(seeds[community]) < community_sizes[community] * community_betweenness[community]:
        #     seeds[community].append(node)
        # if len(seeds[community]) < community_sizes[community] * community_degree[community]:
        #     seeds[community].append(node)
        # if len(seeds[community]) < community_sizes[community] * community_eigen[community]:
        #     seeds[community].append(node)
        if len(seeds[community]) < community_sizes[community] * community_pagerank[community]:
            seeds[community].append(node)

print("Number of seeds:", len(seeds))

In [ ]:
# Create a copy of the original graph to be modified later, by removing nodes (fair version)
G2_copy_comm_fair = G2.copy()


G2_copy_comm_fair.remove_nodes_from(seeds)
print(G2_copy_comm_fair)

In [ ]:
degree_dict = dict(G2_copy_comm_fair.degree())
weights = []
for u, v, data in G2_copy_comm_fair.edges(data=True):
    ratio = 1 / degree_dict[v]
    data['weight'] = ratio
    weights.append(ratio)

print(weights)

In [ ]:
import community
# Calculcate communities using louvain method
partition = community.best_partition(G2_copy_comm_fair)
for node in G2_copy_comm_fair.nodes:
    G2_copy_comm_fair.nodes[node]['community'] = partition[node]
print(len(partition))

num_distinct_values = len(set(partition.values()))

print("The dictionary has", num_distinct_values, "distinct values.")

In [ ]:
community_sizes = {}
for c in set(partition.values()):
    nodes_in_community = [n for n in partition.keys() if partition[n] == c]
    community_sizes[c] = len(nodes_in_community)

# Calculate the centralities for each node in the graph

closeness_centrality = nx.closeness_centrality(G2_copy_comm_fair)
betweenness_centrality = nx.betweenness_centrality(G2_copy_comm_fair)
degree_centrality = nx.degree_centrality(G2_copy_comm_fair)
eigen_centrality = nx.eigenvector_centrality(G2_copy_comm_fair)
pagerank_centrality = nx.pagerank(G2_copy_comm_fair)

In [ ]:
# Create a dictionary to store the population ratio of each community:

# community_sizes = {}
# for c in set(partition.values()):
#     nodes_in_community = [n for n in partition.keys() if partition[n] == c]
#     community_sizes[c] = len(nodes_in_community)

# # Calculate the centralities for each node in the graph

# # closeness_centrality = nx.closeness_centrality(G2_copy_comm_fair)
# # betweenness_centrality = nx.betweenness_centrality(G2_copy_comm_fair)
# degree_centrality = nx.degree_centrality(G2_copy_comm_fair)
# # eigen_centrality = nx.eigenvector_centrality(G2_copy_comm_fair)
# # pagerank_centrality = nx.pagerank(G2_copy_comm_fair)

# Create a dictionary to store the centralities for each community:

community_closeness = {}
community_betweenness = {}
community_degree = {}
community_eigen = {}
community_pagerank = {}
for node in G2_copy_comm_fair.nodes():
    community = partition[node]
    if community not in community_closeness:
        community_closeness[community] = 0
    # if community not in community_betweenness:
    #     community_betweenness[community] = 0
    # if community not in community_degree:
    #     community_degree[community] = 0
    # if community not in community_eigen:
    #     community_eigen[community] = 0
    # if community not in community_pagerank:
    #     community_pagerank[community] = 0

    community_closeness[community] += closeness_centrality[node]
    # community_betweenness[community] += betweenness_centrality[node]
    # community_degree[community] += degree_centrality[node]
    # community_eigen[community] += eigen_centrality[node]
    # community_pagerank[community] += pagerank_centrality[node]


# Calculate the total centrality for each community

for community in community_closeness:
    community_closeness[community] /= community_sizes[community]
# for community in community_betweenness:
#     community_betweenness[community] /= community_sizes[community]
# for community in community_degree:
#     community_degree[community] /= community_sizes[community]
# for community in community_eigen:
#     community_eigen[community] /= community_sizes[community]
# for community in community_pagerank:
#     community_pagerank[community] /= community_sizes[community]

# Calculate the number of seeds for each community proportional to the population ratio using the centralities:
seeds = {}
for node in G2_copy_comm_fair.nodes():
    community = partition[node]
    if community not in seeds:
        seeds[community] = []
        if len(seeds[community]) < community_sizes[community] * community_closeness[community]:
            seeds[community].append(node)
        # if len(seeds[community]) < community_sizes[community] * community_betweenness[community]:
        #     seeds[community].append(node)
        # if len(seeds[community]) < community_sizes[community] * community_degree[community]:
        #     seeds[community].append(node)
        # if len(seeds[community]) < community_sizes[community] * community_eigen[community]:
        #     seeds[community].append(node)
        # if len(seeds[community]) < community_sizes[community] * community_pagerank[community]:
        #     seeds[community].append(node)

print("Number of seeds:", len(seeds))

The new correct implementation for fairness

In [ ]:
import community

# Compute the Louvain communities
partition = community.best_partition(G1)

# Compute the size of each community
community_sizes = np.bincount(list(partition.values()))

# Compute the total number of nodes in the graph
total_nodes = len(G1.nodes())

# Compute the number of communities and print it
num_distinct_values = len(set(partition.values()))

print("The dictionary has", num_distinct_values, "distinct values.")

proportional_sizes = community_sizes / total_nodes

In [ ]:
# Initialize a dictionary to store the seed nodes
seeds = {}
k=100
for community_id in set(partition.values()):
    nodes_in_community = [node for node, comm in partition.items() if comm == community_id]

    # Compute the number of seed nodes for the current community
    num_seeds = int(round(proportional_sizes[community_id] * len(nodes_in_community)))      

    # Sort the nodes in the current community by centrality measure
    # nodes_sorted_by_dc = sorted(nodes_in_community, key=lambda node: degree_centrality[node], reverse=True)
    # nodes_sorted_by_bc = sorted(nodes_in_community, key=lambda node: betweenness[node], reverse=True)
    # nodes_sorted_by_cc = sorted(nodes_in_community, key=lambda node: closeness[node], reverse=True)
    nodes_sorted_by_pc = sorted(nodes_in_community, key=lambda node: pagerank[node], reverse=True)
    # nodes_sorted_by_ec = sorted(nodes_in_community, key=lambda node: eigenvector[node], reverse=True)
    
    # Add the top `num_seeds` nodes to the seed dictionary 
    # seeds.update({node: degree_centrality[node] for node in nodes_sorted_by_dc[:num_seeds]})
    # seeds.update({node: betweenness[node] for node in nodes_sorted_by_bc[:num_seeds]})
    # seeds.update({node: closeness[node] for node in nodes_sorted_by_cc[:num_seeds]})
    seeds.update({node: pagerank[node] for node in nodes_sorted_by_pc[:num_seeds]})
    # seeds.update({node: eigenvector[node] for node in nodes_sorted_by_ec[:num_seeds]})

print(len(seeds))


In [ ]:
print(seeds)

In [ ]:
# Create a copy of the original graph to be modified later, by removing nodes (fair version)
G1_copy_comm_fair = G1.copy()


G1_copy_comm_fair.remove_nodes_from(seeds)
print(G1_copy_comm_fair)

In [ ]:
# Calcualte the centralities for the new graph

degree_after = nx.degree_centrality(G1_copy_comm_fair)
closeness_after = nx.closeness_centrality(G1_copy_comm_fair)
betweenness_after = nx.betweenness_centrality(G1_copy_comm_fair)
degree_after = nx.degree_centrality(G1_copy_comm_fair)
eigen_after = nx.eigenvector_centrality(G1_copy_comm_fair)
pagerank_after = nx.pagerank(G1_copy_comm_fair)

In [ ]:
import community

# Compute the Louvain communities
partition_after = community.best_partition(G1_copy_comm_fair)

# Compute the size of each community

community_sizes_after = np.bincount(list(partition_after.values()))

# Compute the total number of nodes in the graph
total_nodes_after = len(G1_copy_comm_fair.nodes())

# Compute the number of communities and print it
num_distinct_values = len(set(partition_after.values()))

print("The dictionary has", num_distinct_values, "distinct values.")

proportional_sizes_after = community_sizes_after / total_nodes_after

In [ ]:
print(partition_after)

In [ ]:
# Initialize a dictionary to store the seed nodes
seeds_after = {}
for community_id in set(partition_after.values()):
    nodes_in_community_after = [node for node, comm in partition_after.items() if comm == community_id]

    # Compute the number of seed nodes for the current community
    num_seeds_after = int(round(proportional_sizes_after[community_id] * len(nodes_in_community_after)))   
    num_group_seeds = int((num_group_nodes / total_nodes_after) * k)

    # Sort the nodes in the current community by betweenness centrality
    # nodes_sorted_by_dc_after = sorted(nodes_in_community_after, key=lambda node: degree_after[node], reverse=True)
    # nodes_sorted_by_bc_after = sorted(nodes_in_community_after, key=lambda node: betweenness_after[node], reverse=True)
    # nodes_sorted_by_cc_after = sorted(nodes_in_community_after, key=lambda node: closeness_after[node], reverse=True)
    # nodes_sorted_by_pc_after = sorted(nodes_in_community_after, key=lambda node: pagerank_after[node], reverse=True)
    nodes_sorted_by_ec_after = sorted(nodes_in_community_after, key=lambda node: eigen_after[node], reverse=True)
    
    # Add the top `num_seeds` nodes to the seed dictionary 
    # seeds_after.update({node: degree_after[node] for node in nodes_sorted_by_dc_after[:num_seeds_after]})
    # seeds_after.update({node: betweenness_after[node] for node in nodes_sorted_by_bc_after[:num_seeds_after]})
    # seeds_after.update({node: closeness_after[node] for node in nodes_sorted_by_cc_after[:num_seeds_after]})
    # seeds_after.update({node: pagerank_after[node] for node in nodes_sorted_by_pc_after[:num_seeds_after]})
    seeds_after.update({node: eigen_after[node] for node in nodes_sorted_by_ec_after[:num_seeds_after]})

print(len(seeds_after))

In [ ]:
print(len(seeds_after))

For the following code dont forget to change the graphs for before/after blocking the nodes

In [ ]:
# ICM
# This code can be used with the previous two cell blocks to calculate the influence count. 
# It combines the method of calculating the spread using spreading model as defined in the NDlib package, witht the different heuristic approaches.
average = []
for index in range(7):
    model = ep.IndependentCascadesModel(G1)
    
    # Model configuration
    cfg = mc.Configuration()
    
    infected_nodes = seeds
    cfg.add_model_initial_configuration("Infected", infected_nodes)
    model.set_initial_status(cfg)

    # Setting the edge parameters
    for e in G1.edges():
        cfg.add_edge_configuration("threshold", e, random.uniform(0,1))

    list = []
    # Simulation execution
    for i in range(0,1000):
        iterations = model.iteration_bunch(1)

    # Get infected nodes
        dict = iterations[0]
        for i in dict:
            k = (i,dict[i])
            list.append(k)
    list2 = []
    for i in range(len(list)):
        if list[i][0] == 'status':
            list2.append(list[i][1])
    list4 = []
    for t in range(len(list2)):
        for f in list2[t]:
            if list2[t][f] == 1:
                list4.append(f)
 
    percentage = (len(list4)/len(G1.nodes()))*100
    
    average.append(percentage)
print("The average percentage is: ", sum(average)/len(average), "%")

In [ ]:
# LTM
# This code can be used with the previous two cell blocks to calculate the influence count. 
# It combines the method of calculating the spread using spreading model as defined in the NDlib package, witht the different heuristic approaches.
average = []
for index in range(7):
    model = ep.ThresholdModel(G1)
    cfg = mc.Configuration()
    infected_nodes = seeds
    cfg.add_model_initial_configuration("Infected", infected_nodes)
    model.set_initial_status(cfg)
    threshold = 0.1
    for n in G1.nodes():
        cfg.add_node_configuration("threshold", n, random.uniform(0,1))

    list = []
    for i in range(0,1000):
        iterations = model.iteration_bunch(1)

        dict = iterations[0]
        for i in dict:
            k = (i,dict[i])
            list.append(k)
    list2 = []
    for i in range(len(list)):
        if list[i][0] == 'status':
            list2.append(list[i][1])
    list4 = []
    for t in range(len(list2)):
        for f in list2[t]:
            if list2[t][f] == 1:
                list4.append(f)
    percentage = (len(list4)/len(G1.nodes()))*100
    average.append(percentage)
print("The average percentage is: ", sum(average)/len(average), "%")

In [ ]:
# ICM
# This code can be used with the previous two cell blocks to calculate the influence count. 
# It combines the method of calculating the spread using spreading model as defined in the NDlib package, witht the different heuristic approaches.
average = []
for index in range(7):
    model = ep.IndependentCascadesModel(G2_copy)
    # model = ep.SIRModel(G2)
    # model = ep.SIModel(G2)
    # model = ep.ThresholdModel(G2_copy_comm_fair)
    cfg = mc.Configuration()
    # cfg.add_model_parameter('beta', 0.01)
    # cfg.add_model_parameter('gamma', 0.005)
    # cfg.add_model_parameter('lambda', 0.005)
    infected_nodes = first_100_after
    cfg.add_model_initial_configuration("Infected", infected_nodes)
    # cfg.add_model_parameter("fraction_infected", 0.05)
    model.set_initial_status(cfg)
    threshold = 0.1
    for e in G2_copy.edges():
        cfg.add_edge_configuration("threshold", e, 0.1)
    # for n in G2.nodes():
    #     cfg.add_node_configuration("threshold", n, threshold)

    # iterations = model.iteration_bunch(500, True, True)
    # trends = model.build_trends(iterations)
    # viz = DiffusionTrend(model, trends)
    # p = viz.plot(width=800, height=800)
    # show(p)
    list = []
    for i in range(0,50):
        iterations = model.iteration_bunch(200)

        dict = iterations[0]
        for i in dict:
            k = (i,dict[i])
            list.append(k)
    list2 = []
    for i in range(len(list)):
        if list[i][0] == 'status':
            list2.append(list[i][1])
    list4 = []
    for t in range(len(list2)):
        for f in list2[t]:
            if list2[t][f] == 1:
                list4.append(f)
    # print("================================")
    # print(f"Results for iteration {index+1}:")
    # print("Total nodes infected: ", len(list4), "out of", len(G2.nodes()))
    percentage = (len(list4)/len(G2_copy.nodes()))*100
    # print("Percentage of total infected nodes with current seed set: ", percentage, "%")
    # print("================================")
    average.append(percentage)
print("The average percentage is: ", sum(average)/len(average), "%")